<a href="https://colab.research.google.com/github/HarikaSatti06112001/Data-Mining/blob/main/CMPE255_ANN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CMPE255_Assignment


## LSH

In [5]:
!pip install lshashing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#important libraries
import numpy as np
from lshashing import LSHRandom

# Loading the data
lsh_data = np.random.randint(size = (10, 10), low = 0, high = 50)
lsh_point = np.random.randint(size = (1,10), low = 0, high = 50)

# LSH appropriate nearest neighbor search
lshashing = LSHRandom(lsh_data, hash_len = 3, num_tables = 2)

# Displaying data
print(lshashing.tables[1].hash_table)
print(lshashing.knn_search(lsh_data, lsh_point[0], k = 3, buckets = 2, radius = 2))

{0: [0, 3, 4, 5, 6, 8, 9], 1: [1], 4: [2], 9: [7]}
[[53.66563146  1.        ]
 [61.7170965   5.        ]
 [63.4428877   9.        ]]


In [8]:
# Loading the data
lsh_data = np.random.randint(size = (10, 10), low = 0, high = 50)
lsh_point = np.random.randint(size = (1, 10), low = 0, high = 50)

# LSH appropriate nearest neighbor search
lsh_random_parallel = LSHRandom(lsh_data, 4, parallel = True)
lsh_random_parallel.knn_search(lsh_data, lsh_point[0], 5, 2, parallel = True)

[Neighbor(index=8, distance=52.09606511052442, value=[[49 38 21]...]),
 Neighbor(index=6, distance=58.360945845659494, value=[[40  2 32]...]),
 Neighbor(index=1, distance=64.35837163881635, value=[[32 14  6]...]),
 Neighbor(index=0, distance=66.82813778641449, value=[[ 5 35 34]...]),
 Neighbor(index=7, distance=67.59437846448475, value=[[44 26 35]...])]

Exhaustive Search

In [30]:
#Exhaustive Search
# Compute the squared Euclidean distance
def SED(X, Y):
    return sum((i-j)**2 for i, j in zip(X, Y))

SED( (9, 7), (3, 5) )

40

In [31]:
# nearest neighbor with the reference points
def exhaustive_search(*, es_points, exhaustive_points):
    return {
        query_p: min(
            exhaustive_points,
            key=lambda X: SED(X, query_p),
        )
        for query_p in es_points
    }


exhaustive_points = [ (2, 3), (1, 5), (3, 4), (5, 10) ]
es_points = [
    (11, 3), (2, 7), (8, 5), (1, 10), (17, 15), (7, 13), (7, 9)
]

exhaustive_search(
    exhaustive_points = exhaustive_points,
    es_points = es_points,
)

{(11, 3): (3, 4),
 (2, 7): (1, 5),
 (8, 5): (3, 4),
 (1, 10): (5, 10),
 (17, 15): (5, 10),
 (7, 13): (5, 10),
 (7, 9): (5, 10)}

Product Quantization

In [32]:
#Product Quantization
!pip install nanopq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# Import libraries
import nanopq
import numpy as np

# Loading the data
value1 = np.random.random((500, 10)).astype(np.float32)
value2 = np.random.random((250, 10)).astype(np.float32)
pq_query_data = np.random.random((10, )).astype(np.float32)

In [34]:
pq_data = nanopq.PQ(M=2, Ks=10, verbose=True)

M: 2, Ks: 10, code_dtype: <class 'numpy.uint8'>


In [35]:
pq_data = nanopq.PQ(M=2, Ks=10).fit(vecs=value2, iter=20, seed=123)

M: 2, Ks: 10, code_dtype: <class 'numpy.uint8'>
iter: 20, seed: 123
Training the subspace: 0 / 2
Training the subspace: 1 / 2


In [36]:
# Vectors encoded to PQ-codes.
value1_code = pq_data.encode(vecs=value1)

Encoding the subspace: 0 / 2
Encoding the subspace: 1 / 2


In [37]:
pq_datatable = pq_data.dtable(query=pq_query_data) 
new_pq = pq_datatable.adist(codes=value1_code) 
new_pq = pq_data.dtable(query=pq_query_data).adist(codes=value1_code) 
min_value = np.argmin(new_pq)

In [38]:
# results by PQ
print(new_pq[:30])

# results by exact scan
new_pq_exact = np.linalg.norm(value1 - pq_query_data, axis=1) ** 2
print(new_pq_exact[:30])

[1.5657753 1.1788459 1.6985219 2.1477566 2.3589149 1.4196291 2.0016105
 2.006404  2.2181597 1.6985219 1.6985219 2.7178233 2.250536  2.17564
 2.006404  2.5560088 2.6041458 0.9287844 1.4196291 1.9461696 1.9474475
 0.6296216 1.8238709 2.467494  1.5313567 2.17564   2.7178233 1.6809732
 2.1477566 2.2704003]
[2.4165907 1.6422672 1.5660034 2.4318826 2.805343  2.087943  1.7851839
 2.7362037 2.7006736 2.5547798 2.2918859 4.0221395 1.9503996 3.3413293
 2.676378  3.0376074 2.6648512 1.3932891 1.4521463 2.703215  2.5337808
 1.0248097 2.557336  2.747135  2.0924556 1.5734069 2.7561963 2.1583164
 2.7194486 2.3297603]


In [39]:
# Vectors approximately reconstructed by fetching codewords
value1_reconstructed = pq_data.decode(codes=value1_code)
print(value1[:3])
print(value1_reconstructed[:3])

[[0.2422198  0.9376684  0.9080111  0.34879732 0.6346381  0.27384222
  0.20611513 0.33633953 0.3270999  0.8822761 ]
 [0.82230383 0.7096232  0.9593452  0.42254335 0.24503304 0.11739844
  0.30105335 0.14526373 0.0921861  0.6029322 ]
 [0.36418745 0.56457037 0.19133572 0.6769059  0.21550545 0.2780236
  0.74176043 0.5597379  0.33483642 0.5429888 ]]
[[0.51959455 0.771619   0.79991984 0.44911277 0.7566727  0.25094247
  0.3246694  0.26181155 0.2943019  0.53498757]
 [0.6653395  0.6993278  0.7855687  0.6575344  0.23237245 0.25094247
  0.3246694  0.26181155 0.2943019  0.53498757]
 [0.35666847 0.2594796  0.2642518  0.7086582  0.24931538 0.23218478
  0.47132704 0.7131341  0.2418168  0.28841886]]


## Trees and Graphs

In [53]:
# Importing libraries 
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Loading the data
data = np.array([[-11, -5], [-8, -10], [-7, -1], [5, 3], [4, 1], [9, 8]])

# nearest neighbors with ball tree approach
tg_neighbors = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(data)
tg_distances, tg_indices = tg_neighbors.kneighbors(data)
tg_indices
tg_distances

array([[0.        , 5.65685425],
       [0.        , 5.83095189],
       [0.        , 5.65685425],
       [0.        , 2.23606798],
       [0.        , 2.23606798],
       [0.        , 6.40312424]])

In [54]:
tg_neighbors.kneighbors_graph(data).toarray()

array([[1., 0., 1., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 1.]])

## HNSW

In [47]:
!pip install n2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 21.9 MB/s 
  Created wheel for n2: filename=n2-0.1.7-cp37-cp37m-linux_x86_64.whl size=2303099 sha256=a66f1790b6ca964600e2d20a3a2622337cb0e05eb42bd0637b4f2d227870d7ef
  Stored in directory: /root/.cache/pip/wheels/36/da/12/b157ca1c9dcdd5fd3fa5e15b7823f805396fb6e6b30427465a
Successfully built n2


In [55]:
# Importing Libraries 
import numpy as np
from n2 import HnswIndex

# Loading data
Values, reference = 20240, 20
hnsw_data = np.arange(Values * reference).reshape(Values, reference)

# Applying HNSW approach
hnsw_index_data = HnswIndex(reference)
for data in hnsw_data:
    hnsw_index_data.add_data(data)
hnsw_index_data.build(m=5, n_threads=3)
print(hnsw_index_data.search_by_id(15, 36))

[15, 16, 14, 17, 13, 18, 19, 12, 20, 21, 22, 23, 11, 24, 25, 26, 27, 28, 29, 10, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 9, 42, 43, 44]
